<a href="https://colab.research.google.com/github/jx-dohwan/BERT-FP_MRS/blob/main/post_pretrain/Post_Pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Post-training

## 라이브러리 설치

In [1]:
!pip install torch
!pip install transformers==4.25.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## 데이터 다운로드

### 스마일 스타일 데이터세트(for 대화 및 스타일 트랜스퍼)

In [3]:
!git clone https://github.com/smilegate-ai/korean_smile_style_dataset
# https://corpus.korean.go.kr/
# https://aihub.or.kr/aihub-data/natural-language/about

fatal: destination path 'korean_smile_style_dataset' already exists and is not an empty directory.


In [4]:
import pandas as pd
df = pd.read_csv('/content/korean_smile_style_dataset/smilestyle_dataset.tsv', sep='\t')
df.to_csv('./korean_smile_style_dataset/smile.csv', index=False)

In [5]:
!head -3 './korean_smile_style_dataset/smile.csv'

formal,informal,android,azae,chat,choding,emoticon,enfp,gentle,halbae,halmae,joongding,king,naruto,seonbi,sosim,translator
안녕하세요. 저는 고양이 6마리 키워요.,안녕! 나는 고양이 6마리 키워.,휴먼. 반갑다. 안드로이드는. 고양이. 6마리. 소유중.,아이고 안녕하십니까~ 나는 그냥 고양이 6마리 키우고 있는 사람이여,하잉ㅋㅋ 나 떼걸룩 6마리 키운다!,ㅎㅇ 나 주인님 6마리 모심 ㅋㅋ,안녕!! >< 나는 고양이😺를 ➏ 마리 키우고있어!! 0_0,안녕안녕~! 나 고양이 6마리나 키운다? 완전 대박이징~,"안녕하십니까,, 저는 고양이 6마리 키웁니다.",안녕하신가~... 난 지금 고양이를 6마리 키우고 있다네,하유 시벌것 괭이놈 6마리 키우는데 힘들어 죽겟네,안녕하냐 ㅡㅡ 나 씹냥이 6마리나 키운다 하;,반갑소. 짐은 고양이를 6마리나 키우오.,안녕하냐니깐! 난 고양이를 6마리 키우고있다니깐!,안녕하시오! 소인은 고양이를 6마리 키우고 있소!,안녕… 난 고양이 6마리 키워 ㅠㅠ,반가운. 나는 6마리의 고양이를 소지하고 있다.
고양이를 6마리나요? 키우는거 안 힘드세요?,고양이를 6마리나? 키우는거 안 힘들어?,고양이. 6마리. 양육. 번거로운가.,아니 무슨 고양이를 6마리나? 거 키우는 거 안 힘든가?,엥? 6마리나? 안힘듬?ㅋㅋㅋㅋ,6마리? 에바아니냐 안 힘듦?,고양이를 6마리나?!! w(ﾟДﾟ)w 키우는거 안 힘듬?? (⊙_⊙;),고양이를 6마리나? 완전 대박~ 키우는 거 안 힘들어?!,"고양이를 6마리나 키우십니까? 안 힘드신지,,",고양이를 6마리나? 키우는거 힘들지 않는가?,니기럴 털만 날리는 거 키우기 안 힘들데?,아니 고양이를 6마리나? 안힘드냐?,고양이를 6마리나? 키우는게 수고스럽진 않소?,고양이를 6마리나? 키우는거 힘들지 않냐니깐?,고양이를 6마리나 키우고 있는 것이오? 힘들지 않소?,고양이..6마리나? ㅠ 키우는건 혹시 안힘들어..?,6마리의

## 데이터 가공하기

In [6]:
import csv

data_path = './korean_smile_style_dataset/smile.csv'
f = open(data_path, 'r')
rdr = csv.reader(f)

for line in rdr:
    print(line)
    break

['formal', 'informal', 'android', 'azae', 'chat', 'choding', 'emoticon', 'enfp', 'gentle', 'halbae', 'halmae', 'joongding', 'king', 'naruto', 'seonbi', 'sosim', 'translator']


In [7]:
# 데이터 저장
def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])
    return final_data

f = open(data_path, 'r')
rdr = csv.reader(f)

""" 세션 데이터 저장할 것"""
session_dataset = []
session = []

""" 실제 데이터 저장 방식 """
for i, line in enumerate(rdr):
    if i == 0:
        header = line
    else:
        utt = line[0] #formal
        if utt.strip() != '':
            session.append(utt)
        else:
            """ 세션 데이터 저장 """
            session_dataset.append(session)
            session = []
""" 마지막 세션 저장 """
session_dataset.append(session)
f.close()

In [8]:
session_dataset[0]

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?',
 '여섯 살입니다. 갈색 고양이에요.',
 '그럼 가장 어린 고양이가 어떻게 돼요?',
 '한 살입니다. 작년에 분양 받았어요.',
 '그럼 고양이들끼리 안 싸우나요?',
 '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']

In [9]:
len(session_dataset)

236

## 모델을 학습하기 위한 loss 

## MLM을 위한 입력

In [10]:
 from transformers import AutoTokenizer
 tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

In [11]:
tokenizer.special_tokens_map_extended

{'bos_token': '[CLS]',
 'eos_token': '[SEP]',
 'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [12]:
tokenizer.eos_token, tokenizer.sep_token, tokenizer.eos_token_id, tokenizer.sep_token_id

('[SEP]', '[SEP]', 2, 2)

In [13]:
"""
논문 입력 형태 [CLS] u1 [EOU] ... ut-1 [EOU] [SEP] ut [SEP]
sep_token을 다른 토큰으로 변경하자
-> 입력형태 : [CLS] u1 <SEP> ... ut-1 <SEP> [SEP] ut [SEP]
"""

special_tokens = {'sep_token' : '<SEP>'}
tokenizer.add_special_tokens(special_tokens)

1

In [14]:
tokenizer.eos_token, tokenizer.sep_token, tokenizer.eos_token_id, tokenizer.sep_token_id

('[SEP]', '<SEP>', 2, 32000)

In [15]:
tokenizer.mask_token, tokenizer.mask_token_id

('[MASK]', 4)

In [16]:
import random

session = session_dataset[0]
mask_ratio = 0.15
corrupt_tokens = []
output_tokens = []

for i, utt in enumerate(session):
    original_token = tokenizer.encode(utt, add_special_tokens=False)

    mask_num = int(len(original_token)*mask_ratio)
    mask_positions = random.sample([x for x in range(len(original_token))], mask_num)
    corrupt_token = []
    for pos in range(len(original_token)):
        if pos in mask_positions:
            corrupt_token.append(tokenizer.mask_token_id)
        else:
            corrupt_token.append(original_token[pos])
            
    if i == len(session)-1:
        output_tokens += original_token
        corrupt_tokens += corrupt_token
    else:
        output_tokens += original_token + [tokenizer.sep_token_id]
        corrupt_tokens += corrupt_token + [tokenizer.sep_token_id]

In [17]:
print(tokenizer.decode(output_tokens))
print('###')
print(tokenizer.decode(corrupt_tokens))

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에요. <SEP> 그럼 가장 어린 고양이가 어떻게 돼요? <SEP> 한 살입니다. 작년에 분양 받았어요. <SEP> 그럼 고양이들끼리 안 싸우나요? <SEP> 저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.
###
안녕하세요. 저는 고양이 6 [MASK] 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 [MASK]세요 [MASK] <SEP> 제가 워낙 고양이를 좋아해서 [MASK]게 힘들진 않 [MASK]요. <SEP> 가장 나이가 많은 고양이 [MASK] 어떻게 돼요? <SEP> 여섯 살입니다. [MASK] 고양이에요. <SEP> 그럼 가장 [MASK] 고양이가 어떻게 돼요? <SEP> [MASK] 살입니다. 작년에 분양 받았어요. <SEP> 그럼 [MASK]들끼리 안 싸우나요? <SEP> 저희 일곱은 다같이 한 가족입니다 [MASK] 싸우는 일은 없어요 [MASK]


## URC를 위한 입력 토큰

### short session 구성

In [18]:
k =4 # 논문 참조 (3개는 context, 1개는 response)
short_session_dataset = []
for session in session_dataset:
      for i in range(len(session)-k+1):
          short_session_dataset.append(session[i:i+k])
print(len(short_session_dataset))

2762


In [19]:
short_session_dataset[0]

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?']

### URC 입력 만들기

In [20]:
# 모든 발화가 negativ response candidate
import random

all_utts = set()
for session in session_dataset:
    for utt in session:
        all_utts.add(utt)
all_utts = list(all_utts)
print(len(all_utts), all_utts[0])

3430 잠수함이나 스노클링 도구 같이 비싼 도구 없이 어떻게 수면 아래를 감상하나요?


In [21]:
session = short_session_dataset[0]
urc_tokens = []
context_utts = []
for i in range(len(session)):
    utt = session[i]    
    original_token = tokenizer.encode(utt, add_special_tokens=False)
    if i == len(session)-1:
        # urc_tokens += [tokenizer.eos_token_id]
        """ 기존 response 입력 """ # 마지막 발화
        positive_tokens = urc_tokens + original_token 
        """ random negative respons 입력 """ # random으로 선택해서 넣어준다.
        while True:
            random_neg_response = random.choice(all_utts)
            if random_neg_response not in context_utts:
                break
        random_neg_response_token = tokenizer.encode(random_neg_response, add_special_tokens=False)
        random_tokens = urc_tokens + random_neg_response_token
        """ context negative response 입력 """ #context_utts에서 랜덤으로 하나 뽑아준다.
        context_neg_response = random.choice(context_utts)
        context_neg_response_token = tokenizer.encode(context_neg_response, add_special_tokens=False)
        context_neg_tokens = urc_tokens + context_neg_response_token
    else:
        urc_tokens += original_token + [tokenizer.sep_token_id]
    context_utts.append(utt)

In [22]:
print(tokenizer.decode(positive_tokens)) # 0
print('#####')
print(tokenizer.decode(random_tokens)) # 1
print('#####')
print(tokenizer.decode(context_neg_tokens)) # 2

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?
#####
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 저는 출근하는게 너무 즐거워요.
#####
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 안녕하세요. 저는 고양이 6마리 키워요.


## 데이터 로드 만들기

In [23]:
import csv
import random
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class post_loader(Dataset):
    def __init__(self, data_path):
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)

        f = open(data_path, 'r')
        rdr = csv.reader(f)

        """ 세션 데이터 저장할 것"""
        session_dataset = []
        session = []

        """ 실제 데이터 저장 방식 """
        for i, line in enumerate(rdr):
            if i == 0:
                header = line
            else:
                utt = line[0] #formal
                if utt.strip() != '':
                    session.append(utt)
                else:
                    """ 세션 데이터 저장 """
                    session_dataset.append(session)
                    session = []
        """ 마지막 세션 저장 """
        session_dataset.append(session)
        f.close()

        """ short session context """
        self.short_session_dataset = []
        for session in session_dataset:
              for i in range(len(session)-k+1):
                  self.short_session_dataset.append(session[i:i+k])

        """ 모든 발화 저장 """
        self.all_utts = set()
        for session in session_dataset:
            for utt in session:
                self.all_utts.add(utt)
        self.all_utts = list(self.all_utts)


    def __len__(self):
        return len(self.short_session_dataset)

    def __getitem__(self, idx): 
        session = self.short_session_dataset[idx]
        """MLM입력"""
        mask_ratio = 0.15
        self.corrupt_tokens = []
        self.output_tokens = []

        for i, utt in enumerate(session):
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)

            mask_num = int(len(original_token)*mask_ratio)
            mask_positions = random.sample([x for x in range(len(original_token))], mask_num)
            corrupt_token = []
            for pos in range(len(original_token)):
                if pos in mask_positions:
                    corrupt_token.append(self.tokenizer.mask_token_id)
                else:
                    corrupt_token.append(original_token[pos])
                    
            if i == len(session)-1:
                self.output_tokens += original_token
                self.corrupt_tokens += corrupt_token
            else:
                self.output_tokens += original_token + [self.tokenizer.sep_token_id]
                self.corrupt_tokens += corrupt_token + [self.tokenizer.sep_token_id]


        """ label for loss """
        self.corrupt_mask_positions = []
        for pos in range(len(self.corrupt_tokens)):
            if self.corrupt_tokens[pos] == self.tokenizer.mask_token_id:
                self.corrupt_mask_positions.append(pos)

        """ URC 입력 """
        urc_tokens = []
        context_utts = []
        for i in range(len(session)):
            utt = session[i]    
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)
            if i == len(session)-1:
                urc_tokens += [self.tokenizer.eos_token_id]
                """ 기존 response 입력 """ # 마지막 발화
                self.positive_tokens = [self.tokenizer.cls_token_id] + urc_tokens + original_token # cls token 추가 
                """ random negative respons 입력 """ # random으로 선택해서 넣어준다.
                while True:
                    random_neg_response = random.choice(self.all_utts)
                    if random_neg_response not in context_utts:
                        break
                random_neg_response_token = self.tokenizer.encode(random_neg_response, add_special_tokens=False)
                self.random_tokens = [self.tokenizer.cls_token_id] + urc_tokens + random_neg_response_token
                """ context negative response 입력 """ #context_utts에서 랜덤으로 하나 뽑아준다.
                context_neg_response = random.choice(context_utts)
                context_neg_response_token = self.tokenizer.encode(context_neg_response, add_special_tokens=False)
                self.context_neg_tokens = [self.tokenizer.cls_token_id] + urc_tokens + context_neg_response_token
            else:
                urc_tokens += original_token + [self.tokenizer.sep_token_id]
            context_utts.append(utt)

        return self.corrupt_tokens, self.output_tokens, self.corrupt_mask_positions, [self.positive_tokens, self.random_tokens, self.context_neg_tokens], [0,1,2]

In [24]:
data_path = './korean_smile_style_dataset/smile.csv'
post_dataset = post_loader(data_path)

In [25]:
corrupt_tokens, output_tokens, corrupt_mask_positions, urc_inputs, urc_labels = post_dataset[0]

In [26]:
print(post_dataset.tokenizer.decode(corrupt_tokens))
print(post_dataset.tokenizer.decode(output_tokens))
print(corrupt_mask_positions) # 이 부분에 해당하는 loss를 계산해야함함
print('#####')
print(post_dataset.tokenizer.decode(urc_inputs[0]))
print(post_dataset.tokenizer.decode(urc_inputs[1]))
print(post_dataset.tokenizer.decode(urc_inputs[2]))

안녕하 [MASK]. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나 [MASK]? 키우는거 안 [MASK]세요? <SEP> 제 [MASK] 워낙 고양이를 좋아해서 크게 힘들진 않아 [MASK]. <SEP> 가장 나이가 [MASK]은 고양이가 어떻게 돼요?
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?
[2, 18, 24, 29, 40, 46]
#####
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 가장 나이가 많은 고양이가 어떻게 돼요?
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 네, 이제 유전의 기본 원리에 대해 조금 알 것 같아요.
[CLS] 안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> [SEP] 고양이를 6마리나요? 키우는거 안 힘드세요?


## 배치 처리

In [27]:
import csv
import random
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import torch

class post_loader(Dataset):
    def __init__(self, data_path):
        self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
        special_tokens = {'sep_token': '<SEP>'}
        self.tokenizer.add_special_tokens(special_tokens)
        
        f = open(data_path, 'r')
        rdr = csv.reader(f)        
        
        """ 세션 데이터 저장할 것"""
        session_dataset = []
        session = []

        """ 실제 데이터 저장 방식 """
        for i, line in enumerate(rdr):
            if i == 0:
                header  = line
            else:
                utt = line[0]
                if utt.strip() != '':
                    session.append(utt)
                else:
                    """ 세션 데이터 저장 """
                    session_dataset.append(session)
                    session = []
        """ 마지막 세션 저장 """
        session_dataset.append(session)
        f.close()
        
        """ short session context """
        k = 4 # 논문에서 가장 좋았던 숫자
        self.short_session_dataset = []
        for session in session_dataset:    
            for i in range(len(session)-k+1):
                self.short_session_dataset.append(session[i:i+k])
                
        """ 모든 발화 저장 """
        self.all_utts = set()
        for session in session_dataset:
            for utt in session:
                self.all_utts.add(utt)
        self.all_utts = list(self.all_utts)
        
    def __len__(self): # 기본적인 구성
        return len(self.short_session_dataset)
    
    def __getitem__(self, idx): # 기본적인 구성
        session = self.short_session_dataset[idx]
        """ MLM 입력 """
        mask_ratio = 0.15
        self.corrupt_tokens = []
        self.output_tokens = []
        for i, utt in enumerate(session):
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)

            mask_num = int(len(original_token)*mask_ratio)
            mask_positions = random.sample([x for x in range(len(original_token))], mask_num)
            corrupt_token = []
            for pos in range(len(original_token)):
                if pos in mask_positions:
                    corrupt_token.append(self.tokenizer.mask_token_id)
                else:
                    corrupt_token.append(original_token[pos])

            if i == len(session)-1:
                self.output_tokens += original_token
                self.corrupt_tokens += corrupt_token
            else:
                self.output_tokens += original_token + [self.tokenizer.sep_token_id]
                self.corrupt_tokens += corrupt_token + [self.tokenizer.sep_token_id]    
        
        """ label for loss """
        self.corrupt_mask_positions = []
        for pos in range(len(self.corrupt_tokens)):
            if self.corrupt_tokens[pos] == self.tokenizer.mask_token_id:
                self.corrupt_mask_positions.append(pos)                
                
        """ URC 입력 """
        urc_tokens = []
        context_utts = []
        for i in range(len(session)):
            utt = session[i]    
            original_token = self.tokenizer.encode(utt, add_special_tokens=False)
            if i == len(session)-1:
                urc_tokens += [self.tokenizer.eos_token_id]
                """ 기존 response 입력 """
                self.positive_tokens = [self.tokenizer.cls_token_id] + urc_tokens + original_token
                """ random negative respons 입력 """
                while True:
                    random_neg_response = random.choice(self.all_utts)
                    if random_neg_response not in context_utts:
                        break
                random_neg_response_token = self.tokenizer.encode(random_neg_response, add_special_tokens=False)
                self.random_tokens = [self.tokenizer.cls_token_id] + urc_tokens + random_neg_response_token
                """ context negative response 입력 """
                context_neg_response = random.choice(context_utts)
                context_neg_response_token = self.tokenizer.encode(context_neg_response, add_special_tokens=False)
                self.context_neg_tokens = [self.tokenizer.cls_token_id] + urc_tokens + context_neg_response_token
            else:
                urc_tokens += original_token + [self.tokenizer.sep_token_id]
            context_utts.append(utt)
        
        return self.corrupt_tokens, self.output_tokens, self.corrupt_mask_positions, [self.positive_tokens, self.random_tokens, self.context_neg_tokens], [0, 1, 2]
    
    def collate_fn(self, sessions): # 배치를 위한 구성
        '''
            input:
                data: [(session1), (session2), ... ]
            return:
                batch_corrupt_tokens: (B, L) padded
                batch_output_tokens: (B, L) padded
                batch_corrupt_mask_positions: list
                batch_urc_inputs: (B, L) padded
                batch_urc_labels: (B)
                batch_mlm_attentions
                batch_urc_attentions

            batch가 3
            MLM = 3개의 입력데이터 (입력데이터별로 길이가 다름)
            URC = 9개의 입력데이터 (context는 길이가 다름, response candidate도 길이가 다름)
        '''
        batch_corrupt_tokens, batch_output_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_urc_labels = [], [], [], [], []
        batch_mlm_attentions, batch_urc_attentions = [], []
        # MLM, URC 입력에 대해서 가장 긴 입력 길이를 찾기
        corrupt_max_len, urc_max_len = 0, 0
        for session in sessions:
            corrupt_tokens, output_tokens, corrupt_mask_positions, urc_inputs, urc_labels = session
            if len(corrupt_tokens) > corrupt_max_len:
                corrupt_max_len = len(corrupt_tokens)
            positive_tokens, random_tokens, context_neg_tokens = urc_inputs
            if max([len(positive_tokens), len(random_tokens), len(context_neg_tokens)]) > urc_max_len:
                urc_max_len = max([len(positive_tokens), len(random_tokens), len(context_neg_tokens)])
                
        ## padding 토큰을 추가하는 부분
        for session in sessions:
            corrupt_tokens, output_tokens, corrupt_mask_positions, urc_inputs, urc_labels = session
            """ mlm 입력 """
            batch_corrupt_tokens.append(corrupt_tokens + [self.tokenizer.pad_token_id for _ in range(corrupt_max_len-len(corrupt_tokens))])
            batch_mlm_attentions.append([1 for _ in range(len(corrupt_tokens))] + [0 for _ in range(corrupt_max_len-len(corrupt_tokens))])
            
            """ mlm 출력 """
            batch_output_tokens.append(output_tokens + [self.tokenizer.pad_token_id for _ in range(corrupt_max_len-len(corrupt_tokens))])
            
            """ mlm 레이블 """
            batch_corrupt_mask_positions.append(corrupt_mask_positions)
            
            """ urc 입력 """
            # positive_tokens, random_tokens, context_neg_tokens = urc_inputs
            for urc_input in urc_inputs:                            
                batch_urc_inputs.append(urc_input + [self.tokenizer.pad_token_id for _ in range(urc_max_len-len(urc_input))])
                batch_urc_attentions.append([1 for _ in range(len(urc_input))] + [0 for _ in range(urc_max_len-len(urc_input))])
            
            """ urc 레이블 """
            batch_urc_labels += urc_labels
        return torch.tensor(batch_corrupt_tokens), torch.tensor(batch_output_tokens), batch_corrupt_mask_positions, torch.tensor(batch_urc_inputs), \
        torch.tensor(batch_urc_labels), torch.tensor(batch_mlm_attentions), torch.tensor(batch_urc_attentions)

In [28]:
from torch.utils.data import DataLoader
data_path = './korean_smile_style_dataset/smile.csv'
post_dataset = post_loader(data_path)
post_dataloader = DataLoader(post_dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=post_dataset.collate_fn)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [29]:
for data in post_dataloader:
    batch_corrupt_tokens, batch_output_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_urc_labels, batch_mlm_attentions, batch_urc_attentions = data
    break

In [30]:
batch_corrupt_tokens.shape, batch_output_tokens.shape, batch_corrupt_mask_positions, batch_urc_inputs.shape, batch_urc_labels.shape

(torch.Size([2, 50]),
 torch.Size([2, 50]),
 [[1, 14, 28, 33, 46], [4, 16, 28, 35]],
 torch.Size([6, 62]),
 torch.Size([6]))

In [31]:
post_dataset.tokenizer.decode(batch_corrupt_tokens[0,:]), post_dataset.tokenizer.decode(batch_output_tokens[0,:])

('영국 [MASK] 뭘 할지 벌써부터 기대되네요. <SEP> 영국 음식 [MASK] 그렇게 맛있데요. <SEP> 그런가요? 일단 먹고 싶은 [MASK]을 몇 개 정리 [MASK]뒀습니다. <SEP> 그리고 영국의 관광지는 다 보 [MASK]야죠.',
 '영국에서 뭘 할지 벌써부터 기대되네요. <SEP> 영국 음식이 그렇게 맛있데요. <SEP> 그런가요? 일단 먹고 싶은 음식을 몇 개 정리해뒀습니다. <SEP> 그리고 영국의 관광지는 다 보셔야죠.')

In [32]:
post_dataset.tokenizer.decode(batch_corrupt_tokens[0,:]), post_dataset.tokenizer.decode(batch_output_tokens[1,:])

('영국 [MASK] 뭘 할지 벌써부터 기대되네요. <SEP> 영국 음식 [MASK] 그렇게 맛있데요. <SEP> 그런가요? 일단 먹고 싶은 [MASK]을 몇 개 정리 [MASK]뒀습니다. <SEP> 그리고 영국의 관광지는 다 보 [MASK]야죠.',
 '아침을 드시나요? <SEP> 아니요, 저는 아침을 안 먹습니다. <SEP> 저는 매일 아침 시리얼 한 그릇을 먹고 나와요. <SEP> 시리얼 한 그릇을 먹으면 배가 차나요? [PAD] [PAD] [PAD] [PAD] [PAD]')

In [33]:
batch_mlm_attentions

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0]])

In [34]:
post_dataset.tokenizer.decode(batch_urc_inputs[0]), batch_urc_labels

('[CLS] 영국에서 뭘 할지 벌써부터 기대되네요. <SEP> 영국 음식이 그렇게 맛있데요. <SEP> 그런가요? 일단 먹고 싶은 음식을 몇 개 정리해뒀습니다. <SEP> [SEP] 그리고 영국의 관광지는 다 보셔야죠. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 tensor([0, 1, 2, 0, 1, 2]))

## 파일로 저장하기

In [35]:
!touch dataset.py

In [36]:
from dataset import post_loader
from torch.utils.data import DataLoader
data_path = './korean_smile_style_dataset/smile.csv'
post_dataset = post_loader(data_path)
post_dataloader = DataLoader(post_dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=post_dataset.collate_fn)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [37]:
for data in post_dataloader:
    batch_corrupt_tokens, batch_output_tokens, batch_corrupt_mask_positions, batch_urc_inputs, batch_urc_labels, batch_mlm_attentions, batch_urc_attentions = data
    break

In [38]:
batch_corrupt_tokens.shape, batch_output_tokens.shape, batch_corrupt_mask_positions, batch_urc_inputs.shape, batch_urc_labels.shape

(torch.Size([2, 61]),
 torch.Size([2, 61]),
 [[4, 22, 35, 39, 48], [7, 14, 28, 34, 41, 53]],
 torch.Size([6, 89]),
 torch.Size([6]))

## 모델링하기